### 區塊 1: 環境設定與函式庫導入
說明: 建立一個乾淨的 Python 虛擬環境並安裝所需套件。

1. 建立資料夾並進入 (如果尚未建立):
mkdir aicup-final-project
cd aicup-final-project

2. 建立虛擬環境 (venv):
python -m venv venv

3. 啟用虛擬環境:
(Windows): .\\venv\\Scripts\\activate
(macOS/Linux): source venv/bin/activate

4. 安裝必要套件:
pip install pandas numpy scikit-learn lightgbm tqdm

5. 將您的資料檔案放入名為 'data' 的資料夾，然後啟動 Jupyter:
jupyter notebook

In [1]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

print("函式庫導入完成。")

函式庫導入完成。


### 區塊 2: 資料讀取函式

In [36]:
def load_data(dir_path):
    """
    讀取競賽所需的三個核心資料集。
    """
    print("開始讀取資料...")
    paths = {
        "txn": os.path.join(dir_path, 'acct_transaction.csv'),
        "alert": os.path.join(dir_path, 'acct_alert.csv'),
        "predict_list": os.path.join(dir_path, 'acct_predict.csv'),
    }
    df_txn = pd.read_csv(paths['txn'])
    df_alert = pd.read_csv(paths['alert'])
    df_predict_list = pd.read_csv(paths['predict_list'])
    
    print(f"交易紀錄筆數: {len(df_txn)}")
    print(f"警示帳戶名單數: {len(df_alert)}")
    print(f"待預測帳戶數: {len(df_predict_list)}")
    print("資料讀取完成。")
    
    return df_txn, df_alert, df_predict_list

### 區塊 3: 時間窗特徵工程函式

In [37]:
def feature_engineering_by_account(df_txn, df_acct_dates):
    """
    以帳戶為中心，計算時間窗特徵。
    Args:
        df_txn (DataFrame): 原始交易資料表。
        df_acct_dates (DataFrame): 包含 'acct' 和 'date' 欄位的帳戶列表。
    """
    print(f"開始為 {len(df_acct_dates)} 個帳戶進行特徵工程...")
    
    df_txn['txn_amt'] = pd.to_numeric(df_txn['txn_amt'], errors='coerce').fillna(0)
    
    print("  - 正在合併交易資料 (此步驟可能需要數分鐘)...")
    df_merged = df_acct_dates.merge(df_txn, left_on='acct', right_on='from_acct', how='left')
    df_merged = df_merged[df_merged['txn_date'] <= df_merged['date']]
    print("  - 資料合併完成。")
    
    all_features = df_acct_dates[['acct']].copy()
    time_windows = [1, 3, 7, 14, 30]
    
    print("  - 開始計算時間窗特徵...")
    for days in tqdm(time_windows, desc="計算特徵窗口"):
        df_window = df_merged[df_merged['txn_date'] > df_merged['date'] - days]
        
        amt_stats = df_window.groupby('acct')['txn_amt'].agg(['sum', 'count', 'mean', 'std']).add_prefix(f'send_amt_{days}d_')
        all_features = all_features.merge(amt_stats, on='acct', how='left')
        
        unique_to_accts = df_window.groupby('acct')['to_acct'].nunique().rename(f'send_to_unique_accts_{days}d')
        all_features = all_features.merge(unique_to_accts, on='acct', how='left')

    all_features.fillna(0, inplace=True)
    print("特徵工程完成。")
    return all_features

### 區塊 4: 主流程執行

In [ ]:
if __name__ == "__main__":
    
    data_dir = 'data'
    df_txn, df_alert, df_predict_list = load_data(data_dir)
    df_submission_template = pd.read_csv(os.path.join(data_dir, 'submission_template.csv'))
    
    print("準備特徵工程的輸入資料...")
    # 正樣本 (警示帳戶)，並將事件日欄位改名為 'date'
    df_pos_dates = df_alert.rename(columns={'event_date': 'date'})
    
    # 找出所有交易過的帳戶
    all_accts_in_txn = pd.concat([df_txn['from_acct'], df_txn['to_acct']]).unique()
    
    # 找出非警示、也非待預測的帳戶作為負樣本
    non_alert_accts = np.setdiff1d(all_accts_in_txn, pd.concat([df_alert['acct'], df_predict_list['acct']]).unique())
    df_neg_dates = pd.DataFrame({'acct': non_alert_accts})
    last_date = df_txn['txn_date'].max()
    df_neg_dates['date'] = last_date

    # 待預測帳戶 (測試集)，同樣以最後一天為基準日
    df_test_dates = df_predict_list.copy()
    df_test_dates['date'] = last_date

    # 分別為三個群組執行特徵工程
    X_pos_features = feature_engineering_by_account(df_txn, df_pos_dates)
    X_neg_features = feature_engineering_by_account(df_txn, df_neg_dates)
    X_test_features = feature_engineering_by_account(df_txn, df_test_dates)

    print("準備模型訓練資料...")
    X_pos_features['label'] = 1
    X_neg_features['label'] = 0
    
    # 合併正負樣本成為完整的訓練集
    train_df = pd.concat([X_pos_features, X_neg_features]).reset_index(drop=True)
    test_df = X_test_features

    features_columns = [col for col in train_df.columns if col not in ['acct', 'date', 'label']]
    X_train_full = train_df[features_columns]
    y_train_full = train_df['label']
    X_test = test_df[features_columns]
    
    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
    )
    
    print("\n開始使用 LightGBM 進行訓練...")
    model = lgb.LGBMClassifier(
        objective='binary', metric='f1', n_estimators=1000,
        learning_rate=0.05, num_leaves=31, seed=42, n_jobs=-1, verbose=-1
    )
    model.fit(X_train, y_train)
    
    print("\n在驗證集上尋找最佳預測門檻值...")
    y_valid_proba = model.predict_proba(X_valid)[:, 1]
    
    best_f1 = 0
    best_threshold = 0.5
    for threshold in np.arange(0.01, 0.51, 0.01):
        y_valid_pred = (y_valid_proba > threshold).astype(int)
        current_f1 = f1_score(y_valid, y_valid_pred)
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_threshold = threshold
            
    print(f"找到最佳門檻值: {best_threshold:.2f}, 在驗證集上的 F1-Score 為: {best_f1:.4f}")

    print("\n使用全部訓練資料重新訓練模型...")
    final_model = lgb.LGBMClassifier(
        objective='binary', metric='f1', n_estimators=1000,
        learning_rate=0.05, num_leaves=31, seed=42, n_jobs=-1, verbose=-1
    )
    final_model.fit(X_train_full, y_train_full)
    
    print("模型訓練完成。開始使用最佳門檻值進行最終預測...")
    y_pred_proba = final_model.predict_proba(X_test)[:, 1]
    y_pred_final = (y_pred_proba > best_threshold).astype(int)
    print("預測完成。")
    
    df_submission_output = pd.DataFrame({'acct': test_df['acct'], 'label': y_pred_final})
    df_final_submission = df_submission_template[['acct']].merge(df_submission_output, on='acct', how='left')
    df_final_submission['label'] = df_final_submission['label'].fillna(0).astype(int)

    output_path = "submission_final_optimized.csv"
    df_final_submission.to_csv(output_path, index=False)
    
    print(f"\n成功產出提交檔案: {output_path}")
    print(f"預測為警示帳戶 (1) 的數量: {df_final_submission['label'].sum()}")
    print("程式執行完畢。")

開始讀取資料...
交易紀錄筆數: 4435890
警示帳戶名單數: 1004
待預測帳戶數: 4780
資料讀取完成。
準備特徵工程的輸入資料...
開始為 1004 個帳戶進行特徵工程...
  - 正在合併交易資料 (此步驟可能需要數分鐘)...
  - 資料合併完成。
  - 開始計算時間窗特徵...


計算特徵窗口: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 108.69it/s]

特徵工程完成。
開始為 1794322 個帳戶進行特徵工程...
  - 正在合併交易資料 (此步驟可能需要數分鐘)...


  - 資料合併完成。
  - 開始計算時間窗特徵...


計算特徵窗口: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.28s/it]


特徵工程完成。
開始為 4780 個帳戶進行特徵工程...
  - 正在合併交易資料 (此步驟可能需要數分鐘)...
  - 資料合併完成。
  - 開始計算時間窗特徵...


計算特徵窗口: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 27.32it/s]


特徵工程完成。
準備模型訓練資料...

開始使用 LightGBM 進行訓練...
